# Study the WEB (HTML) data crawling and sraping with python

### Get the web info from PubMed with search key-words
* e.g. https://www.ncbi.nlm.nih.gov/pubmed/?term=dry+skin
* Target information:
    * Article Page URL
    * Article title
    * Article Authors
    * Article Abstracts

In [1]:
import urllib.request
# import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd
import numpy as np
import random
#from selenium import webdriver
#import multiprocessing as mp
import re

In [10]:
#Get the maximum page number of search results
def get_Max_page_num(soup):
    max_page_num = int(soup.find('input', {'id':'pageno2'}).get('last'))
    return max_page_num

# #Extract Author information with Multi processing
# def multiCore(url_list):
#     # Multiprocessing
#     p = mp.Pool()

#     for url in url_list:
#         p.apply_async(extractWrite, args=(url,))

#     # Close pool
#     p.close()
#     p.join()

def extract_abstract(url, row):
    # Extract information
    org, keywords, abstract = singlePageExtract(url)
    #PMID
    pmid = url.split('/')[-1]
    
    # Extract title, author, date and other information
    title = row.find('a').get_text()
    author = row.find('p', {'class':'desc'}).get_text()
    journal = row.find('span', {'class':'jrnl'}).get('title')
    date_raw = row.find('p', {'class':'details'}).get_text()
    # Use regular expression to get time information
    date = re.findall(r'\d{4}[\s\w{3}\s\d+]*', date_raw)[0]
    
    #Data
    data = [url, pmid, title, author, journal, date, org, keywords, abstract]
    
    return data

#     # Build data to be loaded into MongoDB
#     data = {
#     'url': link,
#     'title': title,
#     'author': author,
#     'journal': journal,
#     'date': date,
#     'org': org,
#     'keywords': keywords
#     }

#     # Check if already existed
#     if db.med_nlp.find({'url': link}).limit(1):
#     	print(data, 'already exists.')
#     else:
#         # Insert data into MongoDB's Pubmed database's med_nlp collections
#         db.med_nlp.insertOne(data)  

def singlePageExtract(url):
    # Get author's organization and paper keywords
    org = ''
    
    print(url)
    # Set waiting time to avoid high traffic
    #browser.implicitly_wait(random.randint(2, 3))
    
    # Get target page information
    #tempo_html = requests.get(url,  headers = requestHeader(url))
    #tempo_html = urllib.request.Request(url,  headers = requestHeader(url))
    tempo_html = urllib.request.urlopen(url)
    #tempo_soup = BeautifulSoup(tempo_html.text, 'lxml')
    tempo_soup = BeautifulSoup(tempo_html, 'html.parser')
    
    # Find and collect authors' organization
    try:
        collect = tempo_soup.find('dl', {'class':'ui-ncbi-toggler-slave'}).find_all('dd')
        for single in collect:
            org += ';' + single.get_text()
    except:
        org = 'ORGANIZATION_NA'
    
    # Collect article' keywords
    try:
        keywords = tempo_soup.find('div', {'class':'keywords'}).find('p').get_text().split(';')
    except:
        keywords = 'KEYWORDS_NA'
        
    # Collect article's abstract
    try:
        abstract = tempo_soup.find('div', {'class':"abstr"}).get_text()
    except:
        abstract = 'ABSTRACT_NA'
        
#     # Collect journal's information
#     try:
#         journal_info = tempo_soup.find('div', {'class':"cit"}).get_text()
#     except:
#         journal_info = 'JOURNAL_INFO_NA'

    
    return org, keywords, abstract#, journal_info


# def requestHeader(url):
#     # Build request headers
#     headers = {
#             'User-Agent':random.choice(UserAgent),
#             'Referer': url,
#             'Connection':'keep-alive',
#             'Host':'www.ncbi.nlm.nih.gov'
#             }
#     return headers

# UserAgent = [
#         "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
#         "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0",
#         "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36",
#         "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
#         "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
#         "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
#         "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
#         "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
#         "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
#         "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
#         "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
#         "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
#         "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
#         "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
#         "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
#         "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
#         "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
#         "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
#         "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
#         "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
#         "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
#         ]

In [11]:
# def convert_html_to_df(soup):
#     #Storage the contents to dataframe
#     comment_df_column = np.array(["comment_type", "company_name", "comment_category", "comment_data", "comment_status", "comment_time"]) 
#     comment_df = pd.DataFrame(columns = comment_df_column)
    
#     #Type of comment
#     comment_type_list = ['kuchikomi kuchikomi--good kuchikomi--woman', 'kuchikomi kuchikomi--bad kuchikomi--woman', 'kuchikomi kuchikomi--woman', 
#                     'kuchikomi kuchikomi--good kuchikomi--man', 'kuchikomi kuchikomi--bad kuchikomi--man', 'kuchikomi kuchikomi--man']
    
#     #Loop to find target comment type
#     i = 0
#     for comment_type in comment_type_list:
#         comments_all_list = soup.find_all('div', attrs={'class':comment_type})
#         #Check comment exist or not
#         if len(comments_all_list)!=0:
#             for comments_all in comments_all_list:
#                 #Get the coments data
#                 comments = comments_all.find('div', attrs={'class':'comment'})
#                 #Get the company data
#                 company_name = comments.find('a', attrs={'class':'company'}).getText()
#                 company_name = company_name.split('の転職・就職・採用の口コミ情報')[0]
#                 #print(company_name)
#                 #Get the comment category data
#                 comment_category = comments.find('div', attrs={'class':'category'}).getText()
#                 #print(comment_category)
#                 #Get the comment data
#                 comment_data = comments.getText().split('\n\t\t\t\t\t\t\t\t\t')[1]
#                 #print(comment_data)
#                 #Get the comment status data
#                 comment_status = comments.find('div', attrs={'class':'status'}).getText()
#                 #print(comment_status.split('口コミ投稿日：')[0])
#                 #Get the comment time
#                 try: 
#                     comment_time = comment_status.split('口コミ投稿日：')[1]
#                     #print(comment_time)
#                 except:
#                     comment_time = ''
#                     #print(comment_time)
#                 #append the comment data to dataframe
#                 comment_array = [comment_type, company_name, comment_category, comment_data, comment_status.split('口コミ投稿日：')[0], comment_time]
#                 comment_df.loc[i] = comment_array
#                 i = i + 1
#     return comment_df

In [12]:
#Get first page

search_KeyWords = 'dry+skin'

default_url = 'https://www.ncbi.nlm.nih.gov'

search_url = '/pubmed/?term='

urlpage = default_url + search_url + search_KeyWords

#Storage the information
article_data_columns = ['url', 'PMID','title', 'author', 'journal', 'date', 'org', 'keywords', 'abstract']
article_data_df = pd.DataFrame(columns = article_data_columns)
index = 0

#Set up the browser 
#browser = webdriver.Chrome('./chromedriver_macos/chromedriver')

#Get the html data
page = urllib.request.urlopen(urlpage)
soup = BeautifulSoup(page, 'html.parser')

#Get the page url data of each article
all_div = soup.find_all('div', {'class':'rslt'})
#print(len(all_div))
#all_div[0]

# Create/empty url_list to store all the new urls
url_list = []

# Iterate to extract information like author, title, journal, etc
for row_data in all_div[:]:
    # Build article's individual link
    link = default_url + row_data.find('a').get('href')
    #print(link)
    #url_list.append(link)
    
    #Get the single page data
    data = extract_abstract(link, row_data)
    
    #Storage the data
    article_data_df.loc[index] = data
    index = index + 1
    
    #Sleep
    time.sleep(random.randint(1, 3))
    

# Click next buttion to navigate to the next page
#browser.find_element_by_xpath('//*[@title="Next page of results"]').click() 

https://www.ncbi.nlm.nih.gov/pubmed/31371951
https://www.ncbi.nlm.nih.gov/pubmed/31368526
https://www.ncbi.nlm.nih.gov/pubmed/31364497
https://www.ncbi.nlm.nih.gov/pubmed/31361730
https://www.ncbi.nlm.nih.gov/pubmed/31361686
https://www.ncbi.nlm.nih.gov/pubmed/31353514
https://www.ncbi.nlm.nih.gov/pubmed/31342594
https://www.ncbi.nlm.nih.gov/pubmed/31338925
https://www.ncbi.nlm.nih.gov/pubmed/31333023
https://www.ncbi.nlm.nih.gov/pubmed/31332586
https://www.ncbi.nlm.nih.gov/pubmed/31331569
https://www.ncbi.nlm.nih.gov/pubmed/31331315
https://www.ncbi.nlm.nih.gov/pubmed/31329014
https://www.ncbi.nlm.nih.gov/pubmed/31325176
https://www.ncbi.nlm.nih.gov/pubmed/31320871
https://www.ncbi.nlm.nih.gov/pubmed/31318368
https://www.ncbi.nlm.nih.gov/pubmed/31318061
https://www.ncbi.nlm.nih.gov/pubmed/31316015
https://www.ncbi.nlm.nih.gov/pubmed/31309055
https://www.ncbi.nlm.nih.gov/pubmed/31303065


In [13]:
article_data_df

,url,PMID,title,author,journal,date,org,keywords,abstract
0,https://www.ncbi.nlm.nih.gov/pubmed/31371951,31371951,Photothermal conversion hydrogel based mini-ey...,"Pang Y, Wei C, Li R, Wu Y, Liu W, Wang F, Zhan...",International journal of nanomedicine,2019 Jul 11,;Jiangxi Provincial Key Laboratory for Ophthal...,"[gold, hydrogel, nanotechnology, palladium]",AbstractPurpose: The frequent usage of various...
1,https://www.ncbi.nlm.nih.gov/pubmed/31368526,31368526,Enzymatic hydrolysis increases ginsenoside con...,"Kim HW, Han SH, Lee SW, Choi HS, Suh HJ, Hong KB.",Journal of the science of food and agriculture,2019 Aug 1,";Division of Biotechnology, Food Technology, G...","[Cytolase, Enzymatic process, ginsenoside me...",AbstractBACKGROUND: Enzymatic hydrolysis and h...
2,https://www.ncbi.nlm.nih.gov/pubmed/31364497,31364497,Medical compression stockings on the skin mois...,"Westphal T, Konschake W, Haase H, Vollmer M, J...",VASA. Zeitschrift fur Gefasskrankheiten,2019 Jul 31,";1Department of Dermatology, University of Gre...","[Chronic venous insufficiency, compression th...",AbstractBackground: Because of side effects li...
3,https://www.ncbi.nlm.nih.gov/pubmed/31361730,31361730,Solar Radiation Exposure Has Diurnal Effects o...,"Otani H, Goto T, Goto H, Hosokawa Y, Shirato M.",Journal of strength and conditioning research,2019 Jul 29,";Faculty of Health Care Sciences, Himeji Dokky...",KEYWORDS_NA,"AbstractOtani, H, Goto, T, Goto, H, Hosokawa, ..."
4,https://www.ncbi.nlm.nih.gov/pubmed/31361686,31361686,Preclinical Validation of a Novel Device Desig...,"Sammons R, East M, Grant M, Proops D.",Otology & neurotology : official publication o...,2019 Jul 29,";School of Dentistry, University of Birmingham...",KEYWORDS_NA,AbstractAIM: To assess the efficacy of a proto...
5,https://www.ncbi.nlm.nih.gov/pubmed/31353514,31353514,Hydrolyzed collagen from porcine lipase-defatt...,"Chotphruethipong L, Aluko RE, Benjakul S.",Journal of food biochemistry,2019 May,";Faculty of Agro-Industry, Department of Food ...","[antioxidative activity, cell proliferation, ...",AbstractDefatting of seabass skins using porci...
6,https://www.ncbi.nlm.nih.gov/pubmed/31342594,31342594,Solar-induced dorsal skin necrosis in sheep.,"García JA, Romero A, Uzal FA, Tarigo L, Affolt...",Veterinary dermatology,2019 Jul 24,";Centro Universitario Regional Este, Universid...",KEYWORDS_NA,"Abstractin\n\t\t\t\tEnglish, Chinese, 背景: 据报道,..."
7,https://www.ncbi.nlm.nih.gov/pubmed/31338925,31338925,Evaluation of factors triggering sensitive sca...,"Park KH, Kim J, Oh B, Lee E, Hwang-Bo J, Ha J.",Skin research and technology : official journa...,2019 Jul 23,";Skin Research Institute, IEC Korea, Suwon, Ko...","[Korean female, scalp temperature, scalp typ...",AbstractBACKGROUND: The scalp care market is g...
8,https://www.ncbi.nlm.nih.gov/pubmed/31333023,31333023,Spontaneous Generation of Molecular Thin Hydro...,"Choi J, Yoon J, Kim MJ, Pak K, Lee C, Lee H, J...",ACS applied materials & interfaces,2019 Jul 23,ORGANIZATION_NA,KEYWORDS_NA,AbstractPolymer dielectric materials with hydr...
9,https://www.ncbi.nlm.nih.gov/pubmed/31332586,31332586,Magnetic field assisted laser fabrication and ...,"Zhou W, Zhang C, Liu Y, Li Y, Jiang L, Ren L, ...",Biomedical microdevices,2019 Jul 22,;Department of Mechanical & Electrical Enginee...,"[Contact impedance, De-noising, Dry bioelect...",AbstractMagnetic field assisted laser fabricat...


In [89]:
link.split('/')[-1]

'31303065'

In [83]:
soup.find_all('div')

[<div class="grid">
 <div class="col twelve_col nomargin shadow">
 <form action="/pubmed" enctype="application/x-www-form-urlencoded" id="EntrezForm" method="post" name="EntrezForm" onsubmit="return false;">
 <div xmlns:xi="http://www.w3.org/2001/XInclude">
 <!-- no javascript message -->
 <noscript>
 <p class="nojs">
 <strong>Warning:</strong>
 	The NCBI web site requires JavaScript to function. 
 	<a href="/guide/browsers/#enablejs" target="_blank" title="Learn how to enable JavaScript">more...</a>
 </p>
 </noscript>
 <div class="universal_header" id="universal_header" xml:base="http://127.0.0.1/sites/static/header_footer/" xmlns="http://www.w3.org/1999/xhtml"><ul class="inline_list jig-ncbimenu ui-ncbimenu resources_list" id="navcontent"><li class="ui-ncbimenu-item-leaf ui-ncbimenu-item-first ui-helper-reset ui-ncbimenu-item-no-hlt"><a accesskey="1" class="ui-ncbimenu-link-first" href="/" id="ncbihome" role="banner" title="NCBI Home"><span class="offscreen_noflow">NCBI</span><img al

In [77]:
article_data_df["abstract"].values

array(["AbstractPurpose: The frequent usage of various lighting screens has made dry eye syndrome an increasingly serious phenomenon. To relieve this global problem, we have developed a photothermal conversion hydrogel based mini-eye patch. Methods: Gold nanoparticles (GNRs) were synthesized by a seed-mediated method, and then used as the inner cores to grow palladium (Pd) shell by PdCl42-reduction. Then, gelatin was added to prepare GNRs @ Pd hydrogel eye patch by genipin cross-linking. We implanted temperature sensitive ink (complex composed of amino resin and styrene maleic anhydride copolymer) in the eye patch, which could change color at different temperatures. Heating performance of the eye patch was accessed with an infrared temperature profile and the circulating temperature experiment. The safety assessment of the eye patch was conducted by H&E staining of the mouse's eyelid skin and CCK-8 assay. A Keratograph 5M noninvasive ocular surface analyzer was used to assess the impac

In [51]:
row.find('a').get_text()

'Photothermal conversion hydrogel based mini-eye patch for relieving dry eye with long-term use of the light-emitting screen.'

In [32]:
# urllib.request.urlopen(link)

In [34]:
# requests.get(link,  headers = requestHeader(link))

In [43]:
org, keywords, abstract = singlePageExtract(link)

https://www.ncbi.nlm.nih.gov/pubmed/31371951


In [44]:
org

";Jiangxi Provincial Key Laboratory for Ophthalmology, Affiliated Eye Hospital of Nanchang University, Jiangxi Research Institute of Ophthalmology & Visual Science, Nanchang 330006, People's Republic of China.;College of Chemistry, Nanchang University, Nanchang 330088, People's Republic of China.;College of Medicine, Nanchang University, Nanchang 330088, People's Republic of China.;Queen Mary School of Nanchang University, Nanchang University, Nanchang 330088, People's Republic of China.;Institute of Translational Medicine, Nanchang University, Nanchang 330088, People's Republic of China.;Contributed equally"

In [45]:
keywords

['gold', ' hydrogel', ' nanotechnology', ' palladium']

In [46]:
abstract

"AbstractPurpose: The frequent usage of various lighting screens has made dry eye syndrome an increasingly serious phenomenon. To relieve this global problem, we have developed a photothermal conversion hydrogel based mini-eye patch. Methods: Gold nanoparticles (GNRs) were synthesized by a seed-mediated method, and then used as the inner cores to grow palladium (Pd) shell by PdCl42-reduction. Then, gelatin was added to prepare GNRs @ Pd hydrogel eye patch by genipin cross-linking. We implanted temperature sensitive ink (complex composed of amino resin and styrene maleic anhydride copolymer) in the eye patch, which could change color at different temperatures. Heating performance of the eye patch was accessed with an infrared temperature profile and the circulating temperature experiment. The safety assessment of the eye patch was conducted by H&E staining of the mouse's eyelid skin and CCK-8 assay. A Keratograph 5M noninvasive ocular surface analyzer was used to assess the impact of ey

In [61]:
row_data.find('p', {'class':'desc'}).get_text()

'International journal of nanomedicine'

In [52]:
tempo_soup

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head xmlns:xi="http://www.w3.org/2001/XInclude"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- meta -->
<meta content="pubmeddev" name="author"/><meta content="PubMed, National Center for Biotechnology Information, NCBI, United States National Library of Medicine, NLM, MEDLINE, Medical Journals, pub med, Entrez, Journal Articles, Citation search" name="keywords"/><meta content="PubMed comprises more than 29 million citations for biomedical literature from MEDLINE, life science journals, and online books. Citations may include links to full-text content from PubMed Central and publisher web sites." name="description"/><meta content="index,nofollow,noarchive" name="robots"/><meta content="http://www.ncbi.nlm.nih.gov/coreutils/img/pubm

In [14]:
# Iterate to extract information like author, title, journal, etc
for row in all_div:
    # Build article's individual link
    link = default_url + row.find('a').get('href')
    print(link)
    #url_list.append(link)

https://www.ncbi.nlm.nih.gov/pubmed/31371951
https://www.ncbi.nlm.nih.gov/pubmed/31368526
https://www.ncbi.nlm.nih.gov/pubmed/31364497
https://www.ncbi.nlm.nih.gov/pubmed/31361730
https://www.ncbi.nlm.nih.gov/pubmed/31361686
https://www.ncbi.nlm.nih.gov/pubmed/31353514
https://www.ncbi.nlm.nih.gov/pubmed/31342594
https://www.ncbi.nlm.nih.gov/pubmed/31338925
https://www.ncbi.nlm.nih.gov/pubmed/31333023
https://www.ncbi.nlm.nih.gov/pubmed/31332586
https://www.ncbi.nlm.nih.gov/pubmed/31331569
https://www.ncbi.nlm.nih.gov/pubmed/31331315
https://www.ncbi.nlm.nih.gov/pubmed/31329014
https://www.ncbi.nlm.nih.gov/pubmed/31325176
https://www.ncbi.nlm.nih.gov/pubmed/31320871
https://www.ncbi.nlm.nih.gov/pubmed/31318368
https://www.ncbi.nlm.nih.gov/pubmed/31318061
https://www.ncbi.nlm.nih.gov/pubmed/31316015
https://www.ncbi.nlm.nih.gov/pubmed/31309055
https://www.ncbi.nlm.nih.gov/pubmed/31303065


In [4]:
soup

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head xmlns:xi="http://www.w3.org/2001/XInclude"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- meta -->
<meta content="pubmeddev" name="author"/><meta content="PubMed, National Center for Biotechnology Information, NCBI, United States National Library of Medicine, NLM, MEDLINE, Medical Journals, pub med, Entrez, Journal Articles, Citation search" name="keywords"/><meta content="PubMed comprises more than 29 million citations for biomedical literature from MEDLINE, life science journals, and online books. Citations may include links to full-text content from PubMed Central and publisher web sites." name="description"/><meta content="noindex,nofollow,noarchive" name="robots"/><meta content="http://www.ncbi.nlm.nih.gov/coreutils/img/pu